In [ ]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

headers = {'accept': 'application/json'}

In [ ]:
# create an array with the selected tasks.
tasks = []
tasks.append(('semantic-segmentation', 1))
tasks.append(('semantic-segmentation', 2))
tasks.append(('medical-image-segmentation', 1))
tasks.append(('lesion-segmentation', 1))
tasks.append(('brain-tumor-segmentation', 1))
tasks.append(('cell-segmentation', 1))
tasks.append(('retinal-vessel-segmentation', 1))
tasks.append(('3d-medical-imaging-segmentation', 1))

In [ ]:
evaluations = []

for task, page in tasks:
    params = {'page': page}
    url = f'https://paperswithcode.com/api/v1/tasks/{task}/evaluations/'
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        results = response.json()['results']
        print(task, page, len(results))
        for result in results:
            evaluations.append(result['id'])
    else:
        print('err:', response.status_code)

print('tasks        :', len(evaluations))
print('unique tasks :', len(evaluations))

In [ ]:
metrics = []

for evaluation in evaluations:
    url = f'https://paperswithcode.com/api/v1/evaluations/{evaluation}/metrics/'
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        results = response.json()['results']
        metrics.append(results[0]['name'])
    else:
        print('err:', response.status_code)

print('tasks metrics :', len(metrics))

In [ ]:
def count_metrics(mets):
    F1 = 0
    IOU = 0
    OTHER = 0
    for m in mets:
        m = m.lower()
        if 'dice' in m or 'dsc' in m or 'f1' in m:
            F1 += 1
        elif 'iou' in m or 'jaccard' in m:
            IOU += 1
        else:
            OTHER += 1
    return F1, IOU, OTHER
    
F1, IOU, OTHER = count_metrics(metrics)
F1_SS, IOU_SS, OTHER_SS = count_metrics(metrics[:100])
F1_SB, IOU_SB, OTHER_SB = count_metrics(metrics[100:])

print('F1   :', F1)
print('IOU  :', IOU)
print('OTHER:', OTHER)
print('F1   :', F1_SS)
print('IOU  :', IOU_SS)
print('OTHER:', OTHER_SS)
print('F1   :', F1_SB)
print('IOU  :', IOU_SB)
print('OTHER:', OTHER_SB)

In [ ]:
data = []
data.append((F1    , 'F1-Score',  'Tout'))
data.append((F1_SS , 'F1-Score',  'Segmentation Classiques'))
data.append((F1_SB , 'F1-Score',  'Segmentation Biomédicale'))
data.append((IOU   , 'IoU', 'Tout'))
data.append((IOU_SS, 'IoU', 'Segmentation Classiques'))
data.append((IOU_SB, 'IoU', 'Segmentation Biomédicale'))
data.append((OTHER   , 'Autres', 'Tout'))
data.append((OTHER_SS, 'Autres', 'Segmentation Classiques'))
data.append((OTHER_SB, 'Autres', 'Segmentation Biomédicale'))

df = pd.DataFrame(data, columns=['Nombre d\'utilisations', 'Métriques', 'Ensembles'])

plt.figure(figsize=(18,12))
sns.set(font_scale=2.2)
g = sns.barplot(data=df, x="Ensembles", y="Nombre d\'utilisations", hue="Métriques")
g.set_xlabel("Ensembles", labelpad=20)
g.set_ylabel("Count", labelpad=20)

plt.savefig('paperswithcode-metrics-statistics.png')